<a href="https://colab.research.google.com/github/Shamiul-693/detection_of_IoT_botnet_attacks_N_BaIoT-/blob/main/N_BaIoT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle



In [2]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shamiulislam693","key":"7457fbbeb1f26124e027793803020888"}'}

In [3]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle datasets download -d mkashifn/nbaiot-dataset

Dataset URL: https://www.kaggle.com/datasets/mkashifn/nbaiot-dataset
License(s): unknown
 99% 1.73G/1.75G [00:18<00:00, 39.2MB/s]
100% 1.75G/1.75G [00:18<00:00, 100MB/s] 


In [5]:
!unzip /content/nbaiot-dataset.zip

Archive:  /content/nbaiot-dataset.zip
  inflating: 1.benign.csv            
  inflating: 1.gafgyt.combo.csv      
  inflating: 1.gafgyt.junk.csv       
  inflating: 1.gafgyt.scan.csv       
  inflating: 1.gafgyt.tcp.csv        
  inflating: 1.gafgyt.udp.csv        
  inflating: 1.mirai.ack.csv         
  inflating: 1.mirai.scan.csv        
  inflating: 1.mirai.syn.csv         
  inflating: 1.mirai.udp.csv         
  inflating: 1.mirai.udpplain.csv    
  inflating: 2.benign.csv            
  inflating: 2.gafgyt.combo.csv      
  inflating: 2.gafgyt.junk.csv       
  inflating: 2.gafgyt.scan.csv       
  inflating: 2.gafgyt.tcp.csv        
  inflating: 2.gafgyt.udp.csv        
  inflating: 2.mirai.ack.csv         
  inflating: 2.mirai.scan.csv        
  inflating: 2.mirai.syn.csv         
  inflating: 2.mirai.udp.csv         
  inflating: 2.mirai.udpplain.csv    
  inflating: 3.benign.csv            
  inflating: 3.gafgyt.combo.csv      
  inflating: 3.gafgyt.junk.csv       
  inflating:

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,precision_score,confusion_matrix,accuracy_score,precision_recall_fscore_support
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.utils import class_weight

from keras.models import Model
from keras.layers import Input, Conv1D, MaxPool1D, Flatten, Dense
import tensorflow as tf
from tensorflow.keras import callbacks
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.layers import Dense, Dropout,LSTM, Activation,GRU,AveragePooling1D ,BatchNormalization, Reshape,Conv1D, MaxPooling1D,GlobalMaxPooling1D,Embedding

import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from tensorflow.keras import layers



In [7]:
device_info_dataframe=pd.read_csv("/content/device_info.csv")
device_info_dataframe

,DeviceID,DeviceName
0,1,Danmini_Doorbell
1,2,Ecobee_Thermostat
2,3,Ennio_Doorbell
3,4,Philips_B120N10_Baby_Monitor
4,5,Provision_PT_737E_Security_Camera
5,6,Provision_PT_838_Security_Camera
6,7,Samsung_SNH_1011_N_Webcam
7,8,SimpleHome_XCS7_1002_WHT_Security_Camera
8,9,SimpleHome_XCS7_1003_WHT_Security_Camera


In [8]:
Data_summary=pd.read_csv("/content/data_summary.csv")
Data_summary

,File Name,Data Count,Feature Count
0,1.benign.csv,49548,115
1,1.gafgyt.combo.csv,59718,115
2,1.gafgyt.junk.csv,29068,115
3,1.gafgyt.scan.csv,29849,115
4,1.gafgyt.tcp.csv,92141,115
...,...,...,...
84,9.mirai.ack.csv,107187,115
85,9.mirai.scan.csv,43674,115
86,9.mirai.syn.csv,122479,115
87,9.mirai.udp.csv,157084,115


In [9]:
benign_df = pd.read_csv('/content/1.benign.csv')
m_a_df = pd.read_csv('/content/1.mirai.ack.csv')
m_sc_df = pd.read_csv('/content/1.mirai.scan.csv')
m_sy_df = pd.read_csv('/content/1.mirai.syn.csv')
m_u_df = pd.read_csv('/content/1.mirai.udp.csv')
m_u_p_df = pd.read_csv('/content/1.mirai.udpplain.csv')
g_c_df = pd.read_csv('/content/1.gafgyt.combo.csv')
g_j_df = pd.read_csv('/content/1.gafgyt.junk.csv')
g_s_df = pd.read_csv('/content/1.gafgyt.scan.csv')
g_t_df = pd.read_csv('/content/1.gafgyt.tcp.csv')
g_u_df = pd.read_csv('/content/1.gafgyt.udp.csv')
benign_df1 = pd.read_csv('/content/2.benign.csv')
m_a_df1 = pd.read_csv('/content/2.mirai.ack.csv')
m_sc_df1 = pd.read_csv('/content/2.mirai.scan.csv')
m_sy_df1 = pd.read_csv('/content/2.mirai.syn.csv')
m_u_df1 = pd.read_csv('/content/2.mirai.udp.csv')
m_u_p_df1 = pd.read_csv('/content/2.mirai.udpplain.csv')
g_c_df1 = pd.read_csv('/content/2.gafgyt.combo.csv')
g_j_df1 = pd.read_csv('/content/2.gafgyt.junk.csv')
g_s_df1 = pd.read_csv('/content/2.gafgyt.scan.csv')
g_t_df1 = pd.read_csv('/content/2.gafgyt.tcp.csv')
g_u_df1 = pd.read_csv('/content/2.gafgyt.udp.csv')

In [10]:
benign_df['Attack_Class'] = 'benign     '
m_u_df['Attack_Class']    = 'mirai_udp'
m_a_df['Attack_Class']    = 'mirai_ack'
m_sc_df['Attack_Class']   = 'mirai_scan'
m_sy_df['Attack_Class']   = 'mirai_syn'
m_u_p_df['Attack_Class']  = 'mirai_udpplain'
g_c_df['Attack_Class']    = 'gafgyt_combo'
g_j_df['Attack_Class']    = 'gafgyt_junk'
g_s_df['Attack_Class']    = 'gafgyt_scan'
g_t_df['Attack_Class']    = 'gafgyt_tcp'
g_u_df['Attack_Class']    = 'gafgyt_udp'
benign_df1['Attack_Class'] = 'benign     '
m_u_df1['Attack_Class']    = 'mirai_udp'
m_a_df1['Attack_Class']    = 'mirai_ack'
m_sc_df1['Attack_Class']   = 'mirai_scan'
m_sy_df1['Attack_Class']   = 'mirai_syn'
m_u_p_df1['Attack_Class']  = 'mirai_udpplain'
g_c_df1['Attack_Class']    = 'gafgyt_combo'
g_j_df1['Attack_Class']    = 'gafgyt_junk'
g_s_df1['Attack_Class']    = 'gafgyt_scan'
g_t_df1['Attack_Class']    = 'gafgyt_tcp'
g_u_df1['Attack_Class']    = 'gafgyt_udp'

In [11]:
df = pd.concat([benign_df,
                m_u_df, m_a_df, m_sc_df,m_sy_df, m_u_p_df,
                g_c_df,g_j_df, g_s_df, g_t_df,g_u_df,
                benign_df1,m_u_df1, m_a_df1, m_sc_df1,m_sy_df1, m_u_p_df1,
                g_c_df1,g_j_df1, g_s_df1, g_t_df1,g_u_df1],
                axis=0, sort=False, ignore_index=True)

In [12]:
df.head(5)

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc,Attack_Class
0,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,...,0.0,0.0,1.000000,60.000000,0.000000,60.000000,0.000000,0.0,0.0,benign
1,1.000000,354.000000,0.000000,1.000000,354.000000,0.000000,1.000000,354.000000,0.000000,1.000000,...,0.0,0.0,5.319895,344.262695,4.710446,344.262695,22.188299,0.0,0.0,benign
2,1.857879,360.458980,35.789338,1.912127,360.275733,35.923972,1.969807,360.091968,35.991542,1.996939,...,0.0,0.0,6.318264,347.703087,9.034660,347.703087,81.625077,0.0,0.0,benign
3,1.000000,337.000000,0.000000,1.000000,337.000000,0.000000,1.000000,337.000000,0.000000,1.000000,...,0.0,0.0,1.000000,337.000000,0.000000,337.000000,0.000000,0.0,0.0,benign
4,1.680223,172.140917,18487.448750,1.793580,182.560279,18928.175300,1.925828,193.165753,19153.795810,1.992323,...,0.0,0.0,1.000000,60.000000,0.000000,60.000000,0.000000,0.0,0.0,benign


In [13]:
df.describe()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_radius,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc
count,1.854174e+06,1.854174e+06,1.854174e+06,1.854174e+06,1.854174e+06,1.854174e+06,1.854174e+06,1.854174e+06,1.854174e+06,1.854174e+06,...,1.854174e+06,1.854174e+06,1.854174e+06,1.854174e+06,1.854174e+06,1.854174e+06,1.854174e+06,1.854174e+06,1.854174e+06,1.854174e+06
mean,9.624249e+01,2.052305e+02,1.843373e+04,1.532890e+02,2.054967e+02,2.049003e+04,4.409556e+02,2.059617e+02,2.205870e+04,4.156145e+03,...,1.661179e+02,-1.204834e+01,-5.977208e-04,1.079786e+03,2.059568e+02,8.458999e-01,2.085748e+02,2.601214e+02,-1.209806e+01,-4.992637e-04
std,6.614292e+01,1.752707e+02,2.445885e+04,1.047999e+02,1.694320e+02,2.543059e+04,3.010989e+02,1.649477e+02,2.616546e+04,2.839076e+03,...,3.616173e+03,5.545478e+02,1.843718e-02,4.701589e+03,2.217342e+02,1.196102e+01,2.215277e+02,4.892830e+03,9.957348e+02,1.901132e-02
min,1.000000e+00,6.000000e+01,0.000000e+00,1.000000e+00,6.000000e+01,0.000000e+00,1.000000e+00,6.000000e+01,0.000000e+00,1.000000e+00,...,0.000000e+00,-4.005854e+04,-1.010724e+00,1.000000e+00,6.000000e+01,0.000000e+00,6.000000e+01,0.000000e+00,-3.946335e+04,-6.909109e-01
25%,5.651449e+00,6.000000e+01,9.549694e-12,9.922579e+00,6.000003e+01,7.703394e-07,3.388566e+01,6.000441e+01,2.994025e-02,6.591560e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,6.000000e+01,0.000000e+00,6.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.119862e+02,7.405775e+01,3.912630e+01,1.883428e+02,7.407237e+01,4.195164e+01,5.938914e+02,7.417800e+01,4.477901e+01,5.772012e+03,...,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,7.400000e+01,0.000000e+00,7.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.493385e+02,3.696621e+02,4.401520e+04,2.360834e+02,3.788219e+02,4.862206e+04,6.655166e+02,3.882432e+02,5.171356e+04,6.418587e+03,...,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,5.540000e+02,0.000000e+00,5.540000e+02,0.000000e+00,0.000000e+00,0.000000e+00
max,4.380028e+02,1.108155e+03,2.721017e+05,5.058053e+02,1.090333e+03,2.693159e+05,1.185250e+03,9.404513e+02,2.416279e+05,8.912649e+03,...,4.340505e+05,6.036894e+04,9.560988e-01,3.000666e+04,9.090000e+02,3.216536e+02,1.278265e+03,4.793690e+05,1.365841e+05,1.531981e+00


In [14]:
df.shape

(1854174, 116)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1854174 entries, 0 to 1854173
Columns: 116 entries, MI_dir_L5_weight to Attack_Class
dtypes: float64(115), object(1)
memory usage: 1.6+ GB


In [16]:
df.columns

Index(['MI_dir_L5_weight', 'MI_dir_L5_mean', 'MI_dir_L5_variance',
       'MI_dir_L3_weight', 'MI_dir_L3_mean', 'MI_dir_L3_variance',
       'MI_dir_L1_weight', 'MI_dir_L1_mean', 'MI_dir_L1_variance',
       'MI_dir_L0.1_weight',
       ...
       'HpHp_L0.1_covariance', 'HpHp_L0.1_pcc', 'HpHp_L0.01_weight',
       'HpHp_L0.01_mean', 'HpHp_L0.01_std', 'HpHp_L0.01_magnitude',
       'HpHp_L0.01_radius', 'HpHp_L0.01_covariance', 'HpHp_L0.01_pcc',
       'Attack_Class'],
      dtype='object', length=116)

In [18]:
list(df.columns)

['MI_dir_L5_weight',
 'MI_dir_L5_mean',
 'MI_dir_L5_variance',
 'MI_dir_L3_weight',
 'MI_dir_L3_mean',
 'MI_dir_L3_variance',
 'MI_dir_L1_weight',
 'MI_dir_L1_mean',
 'MI_dir_L1_variance',
 'MI_dir_L0.1_weight',
 'MI_dir_L0.1_mean',
 'MI_dir_L0.1_variance',
 'MI_dir_L0.01_weight',
 'MI_dir_L0.01_mean',
 'MI_dir_L0.01_variance',
 'H_L5_weight',
 'H_L5_mean',
 'H_L5_variance',
 'H_L3_weight',
 'H_L3_mean',
 'H_L3_variance',
 'H_L1_weight',
 'H_L1_mean',
 'H_L1_variance',
 'H_L0.1_weight',
 'H_L0.1_mean',
 'H_L0.1_variance',
 'H_L0.01_weight',
 'H_L0.01_mean',
 'H_L0.01_variance',
 'HH_L5_weight',
 'HH_L5_mean',
 'HH_L5_std',
 'HH_L5_magnitude',
 'HH_L5_radius',
 'HH_L5_covariance',
 'HH_L5_pcc',
 'HH_L3_weight',
 'HH_L3_mean',
 'HH_L3_std',
 'HH_L3_magnitude',
 'HH_L3_radius',
 'HH_L3_covariance',
 'HH_L3_pcc',
 'HH_L1_weight',
 'HH_L1_mean',
 'HH_L1_std',
 'HH_L1_magnitude',
 'HH_L1_radius',
 'HH_L1_covariance',
 'HH_L1_pcc',
 'HH_L0.1_weight',
 'HH_L0.1_mean',
 'HH_L0.1_std',
 'HH_L0.1

In [19]:
df.columns.values

array(['MI_dir_L5_weight', 'MI_dir_L5_mean', 'MI_dir_L5_variance',
       'MI_dir_L3_weight', 'MI_dir_L3_mean', 'MI_dir_L3_variance',
       'MI_dir_L1_weight', 'MI_dir_L1_mean', 'MI_dir_L1_variance',
       'MI_dir_L0.1_weight', 'MI_dir_L0.1_mean', 'MI_dir_L0.1_variance',
       'MI_dir_L0.01_weight', 'MI_dir_L0.01_mean',
       'MI_dir_L0.01_variance', 'H_L5_weight', 'H_L5_mean',
       'H_L5_variance', 'H_L3_weight', 'H_L3_mean', 'H_L3_variance',
       'H_L1_weight', 'H_L1_mean', 'H_L1_variance', 'H_L0.1_weight',
       'H_L0.1_mean', 'H_L0.1_variance', 'H_L0.01_weight', 'H_L0.01_mean',
       'H_L0.01_variance', 'HH_L5_weight', 'HH_L5_mean', 'HH_L5_std',
       'HH_L5_magnitude', 'HH_L5_radius', 'HH_L5_covariance', 'HH_L5_pcc',
       'HH_L3_weight', 'HH_L3_mean', 'HH_L3_std', 'HH_L3_magnitude',
       'HH_L3_radius', 'HH_L3_covariance', 'HH_L3_pcc', 'HH_L1_weight',
       'HH_L1_mean', 'HH_L1_std', 'HH_L1_magnitude', 'HH_L1_radius',
       'HH_L1_covariance', 'HH_L1_pcc', 'HH_L0.

##Exploration Data Analysis

In [20]:
df.isnull().sum()   # Identifying Missing Values

,0
MI_dir_L5_weight,0
MI_dir_L5_mean,0
MI_dir_L5_variance,0
MI_dir_L3_weight,0
MI_dir_L3_mean,0
...,...
HpHp_L0.01_magnitude,0
HpHp_L0.01_radius,0
HpHp_L0.01_covariance,0
HpHp_L0.01_pcc,0


In [24]:
null_percentage_per_column = df.isnull().sum()/len(df) * 100
print(null_percentage_per_column)


MI_dir_L5_weight         0.0
MI_dir_L5_mean           0.0
MI_dir_L5_variance       0.0
MI_dir_L3_weight         0.0
MI_dir_L3_mean           0.0
                        ... 
HpHp_L0.01_magnitude     0.0
HpHp_L0.01_radius        0.0
HpHp_L0.01_covariance    0.0
HpHp_L0.01_pcc           0.0
Attack_Class             0.0
Length: 116, dtype: float64
